In [1]:

import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_sampled'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key',
       'spec.index.leaf_size_in_pages', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


In [6]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "btree256": 256,
    "btree1024": 1024,
    "btree2048": 2048,
    "sampledpgm256": 256,
    "sampledpgm1024": 1024,
    "sampledpgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def get_epsilon(x):
    for key,eps in epsilon.items():
        if key in x:
            return eps
    return eps

def indexType(x):
    if "sampledpgm" in x:
        return "SAMPLED_PGM"
    if "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    elif "sj" in x:
        return "SORT_JOIN"
    return "NA"


get_epsilon('samm')
df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['epsilon'] = df['spec.algo_name'].map(lambda x: get_epsilon(x))

print(df['join_algo'].unique())


['inlj']


In [7]:
def plot_for_dataset(dataset, join_algo):
    rows = duckdb.sql(
        " SELECT ratio, index_type, epsilon, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND join_algo='{join_algo}'"
        " GROUP BY ratio, join_algo, index_type, epsilon"
        " ORDER BY ratio"
    ).df()
    rows = rows.pivot(index=['ratio', 'epsilon'], values='v', columns=['index_type'])
    return rows


# INLJ (Raw Lookup)

In [8]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'inlj'))

'fb'

,index_type
ratio,epsilon


'wiki'

,index_type
ratio,epsilon


'books'

...

,index_type
ratio,epsilon


'normal'

,index_type
ratio,epsilon


'lognormal'

,index_type
ratio,epsilon
